# One Pixel Attack Tutorial
## Part 2 - ImageNet

### Dan Kondratyuk
### March 22, 2018

[Part 1](1_one-pixel-attack-cifar10.ipynb)

This time around, we will demonstrate the one pixel attack on ImageNet. The images are 224x224 px in size, with 1000 classes to pick from.

## Imports

Ensure that you have `numpy`, `pandas`, `scipy`, `matplotlib`, `tensorflow-gpu`, and `keras` installed.

In [1]:
# Python Libraries
from __future__ import division, print_function, absolute_import
%matplotlib inline
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import keras
from keras import backend as K
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.imagenet_utils import decode_predictions, preprocess_input

# Helper functions
import helper
from attack import PixelAttacker

matplotlib.style.use('ggplot')
np.random.seed(100)

Using TensorFlow backend.


## Load Dataset

### Download ImageNet Images

To begin, we will demonstrate an attack on one image. The code below will download images from the specified URLs and save to the `data` directory. You may also add more ImageNet URLs, which can be obtained from http://image-net.org/download-imageurls.

In [2]:
with open('data/imagenet_classes.pkl', 'rb') as f:
    class_names = pickle.load(f)
word_to_class = {w:i for i,w in enumerate(class_names)}

In [3]:
# Image URLs given by ImageNet
urls = [
    'http://farm4.static.flickr.com/3154/2585091536_78f528fdee.jpg',
]

# Where each file will be saved to
filenames = [
    'data/balloon.jpg'
]

# The labels of each corresponding image
labels = [
    word_to_class['balloon']
]

In [4]:
# Download all image files
for url, filename in zip(urls, filenames):
    print('Downloading', filename)
    helper.download_from_url(url, filename)

7.87kB [00:00, 78.6kB/s]

122kB [00:01, 109kB/s]  


Let's see what the image looks like. The code below will load an image of a balloon into memory and convert it into a `numpy` array.

In [5]:
originals = [load_img(filename, target_size=(224, 224)) for filename in filenames] # Load an image in PIL format
images = np.array([img_to_array(original) for original in originals]) # Convert the PIL image to a numpy array

In [6]:
helper.plot_image(images[0])

AttributeError: 'module' object has no attribute 'to_rgba'

## Load Model

Keras conveniently supplies pretrained weights for several ImageNet models. We will use `MobileNet` (~30 MB). Running the cell below will initiate a download of the model weights if it is not cached already.

In [7]:
model = keras.applications.MobileNet()

17235968/17225924 [==============================] - 4s 0us/step


Now, predict the image's class.

In [8]:
processed_images = preprocess_input(images.copy()) # Prepare the image for the model
predictions = model.predict(processed_images) # Get the predicted probabilities for each class
label = decode_predictions(predictions) # Convert the probabilities to class labels

label

49152/35363 [=========================================] - 0s 2us/step


[[(u'n02782093', u'balloon', 0.99706227),
  (u'n03888257', u'parachute', 0.0022516712),
  (u'n03720891', u'maraca', 0.00042275016),
  (u'n03443371', u'goblet', 5.517233e-05),
  (u'n04409515', u'tennis_ball', 4.375243e-05)]]

We see that the model correctly predicts `balloon` with 99.7% confidence.

## Perturb Image

Like before, we can perturb an image by changing the color of one or more pixels. The cell below will modify the center pixel to be yellow. As the images are much larger in size than in Cifar10, this change will be barely noticable.

In [9]:
pixel = np.array([112, 112, 255, 255, 0]) # pixel = x,y,r,g,b
image_perturbed = helper.perturb_image(pixel, images)
helper.plot_image(image_perturbed[0])

AttributeError: 'module' object has no attribute 'to_rgba'

Let's see what the model now predicts.

In [10]:
preprocessed_perturbed = preprocess_input(image_perturbed.copy())
predictions = model.predict(preprocessed_perturbed)
label = decode_predictions(predictions)

label

[[(u'n02782093', u'balloon', 0.99702746),
  (u'n03888257', u'parachute', 0.0022734888),
  (u'n03720891', u'maraca', 0.00042949448),
  (u'n03443371', u'goblet', 5.697732e-05),
  (u'n03134739', u'croquet_ball', 4.447125e-05)]]

The model still predicts balloon, but the perturbation did manage to change the confidence slightly.

## Attack

Finally, we use differential evolution to conduct the attack to see what percentage of images we can successfully cause the model to misclassify in a reasonable amount of time.

It is recommended to use Keras with a GPU enabled. The code below can be used to check (if using TensorFlow).

In [11]:
# Should output /device:GPU:0
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [12]:
models = [model]

test = processed_images, np.array([labels])

attacker = PixelAttacker(models, test, class_names, dimensions=(224, 224))

Evaluating mobilenet_1.00_224


In [16]:
result = attacker.attack(0, model, maxiter=20, verbose=True)

Confidence: 0.9961302
Confidence: 0.9961302
Confidence: 0.9961302
Confidence: 0.9961302
Confidence: 0.9961302
Confidence: 0.9961302
Confidence: 0.9961302
Confidence: 0.9961217
Confidence: 0.9961217
Confidence: 0.9961217
Confidence: 0.9961217
Confidence: 0.99567753
Confidence: 0.99567753
Confidence: 0.99567753
Confidence: 0.99567753
Confidence: 0.99567753
Confidence: 0.99567753
Confidence: 0.99567753
Confidence: 0.99567753
Confidence: 0.9956196


In this instance, the attack was not successful. The model still correctly predicts balloon. However, the above output shows that the attack does indeed lower the confidence of the model. If we have a larger dataset, we can see more easily how many of the images in the dataset can be attacked successfully.

### TODO: Attack with more images

In [14]:
# predictions = model.predict(attack_image)
# label = decode_predictions(predictions)
# label